In [1]:
import duckdb
import pandas as pd

In [2]:
con = duckdb.connect()

In [3]:
#Import two tables as SQL tables using any SQL-related Python library.
con.execute("""
CREATE TABLE products_day1 AS
SELECT * FROM read_csv_auto('products_day1.csv', header=True);
""")

con.execute("""
CREATE TABLE products_day2 AS
SELECT * FROM read_csv_auto('products_day2.csv', header=True);
""")

# Optional: Check one of the tables
print(con.execute("SELECT * FROM products_day1").fetchdf())

   product_id               name     category  price  stock
0         101     Wireless Mouse  Electronics  25.99    120
1         102       Water Bottle      Kitchen  10.50    200
2         103  Bluetooth Speaker  Electronics  49.99     75
3         104           Yoga Mat      Fitness  20.00     60
4         105          Desk Lamp    Furniture  35.00     90
5         106           Notebook   Stationery   2.99    300


-------------------------------------------------------------------------------
-------------------------------------------------------------------------------

#Part 1

##Solution 1 for Part 1, Task 1(Identify full rows that were ADDED or REMOVED) based on all columns check

In [4]:
added_removed_query = """
      -- Truly removed rows
          SELECT d1.*, 'REMOVED' AS change_type
          FROM products_day1 d1
          WHERE NOT EXISTS (
              SELECT 1 FROM products_day2 d2
              WHERE d1.name = d2.name
                AND d1.category = d2.category
                AND d1.price = d2.price
                AND d1.stock = d2.stock
          )
          AND d1.product_id NOT IN (
              SELECT d1a.product_id
              FROM products_day1 d1a
              JOIN products_day2 d2a ON d1a.product_id = d2a.product_id
          )

          UNION ALL

      -- Truly added rows
          SELECT d2.*, 'ADDED' AS change_type
          FROM products_day2 d2
          WHERE NOT EXISTS (
              SELECT 1 FROM products_day1 d1
              WHERE d1.name = d2.name
                AND d1.category = d2.category
                AND d1.price = d2.price
                AND d1.stock = d2.stock
          )
          AND d2.product_id NOT IN (
              SELECT d2a.product_id
              FROM products_day2 d2a
              JOIN products_day1 d1a ON d2a.product_id = d1a.product_id
          );
"""
added_removed_df = con.execute(added_removed_query).df()
print(added_removed_df)

   product_id         name    category  price  stock change_type
0         106     Notebook  Stationery   2.99    300     REMOVED
1         107  Smart Watch   Wearables  99.00     50       ADDED


 -------------------------------------------------------------------------------

##Solution 2 for Part 1, Task 1(Identify full rows that were ADDED or REMOVED) based on product_id check alone

In [5]:
added_removed_query = """
      with modified_rows as(
          (SELECT product_id, 'REMOVED' AS change_type FROM products_day1
          EXCEPT
          SELECT product_id, 'REMOVED' AS change_type FROM products_day2)

          UNION ALL

          (SELECT product_id, 'ADDED' AS change_type FROM products_day2
          EXCEPT
          SELECT product_id, 'ADDED' AS change_type FROM products_day1)
      )
      select b.*,change_type from modified_rows a
      join products_day1 b
      on a.product_id = b.product_id

      union all

      select b.*,change_type from modified_rows a
      join products_day2 b
      on a.product_id = b.product_id
"""
added_removed_df = con.execute(added_removed_query).df()
print(added_removed_df)

   product_id         name    category  price  stock change_type
0         106     Notebook  Stationery   2.99    300     REMOVED
1         107  Smart Watch   Wearables  99.00     50       ADDED


###Alternative way for above solution 2

In [6]:
added_removed_query = """
      with modified_rows as(
          --change_type = REMOVED
          (SELECT product_id  FROM products_day1
          EXCEPT
          SELECT product_id FROM products_day2)

          UNION ALL
          -- change_type = 'ADDED'
          (SELECT product_id FROM products_day2
          EXCEPT
          SELECT product_id FROM products_day1)
      )
      select
      *,'REMOVED' as change_type from products_day1
      where product_id in (select product_id from modified_rows )

      union all

      select
      *,'ADDED' as change_type from products_day2
      where product_id in (select product_id from modified_rows)
      order by 1
"""
added_removed_df = con.execute(added_removed_query).df()
print(added_removed_df)

   product_id         name    category  price  stock change_type
0         106     Notebook  Stationery   2.99    300     REMOVED
1         107  Smart Watch   Wearables  99.00     50       ADDED


In [7]:
#Loading resulted output into csv file
added_removed_df.to_csv('Added_removed_rows.csv', index=False)

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------

#Part 2

In [8]:
#Collecting all the list of columns to cehck for here I have taken all columns excluding the Product_Id, which we use for row match check and can be treated as don't change entity

column_query = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'products_day1'
  AND column_name != 'product_id'
ORDER BY ordinal_position
"""

columns = [row[0] for row in con.execute(column_query).fetchall()]

print(columns)


['name', 'category', 'price', 'stock']


In [9]:
# Prepare list to collect results
diff_rows = []

In [10]:
#Generating individual dataframes based on each column uniqueness check
for col in columns:
    query = f"""
        SELECT
            d1.product_id,
            '{col}' AS column_changed,
            CAST(d1.{col} AS TEXT) AS old_value,
            CAST(d2.{col} AS TEXT) AS new_value
        FROM products_day1 d1
        JOIN products_day2 d2 ON d1.product_id = d2.product_id
        WHERE d1.{col} IS DISTINCT FROM d2.{col}
    """
    result = con.execute(query).fetchdf()
    print(result)
    diff_rows.append(result)

Empty DataFrame
Columns: [product_id, column_changed, old_value, new_value]
Index: []
Empty DataFrame
Columns: [product_id, column_changed, old_value, new_value]
Index: []
   product_id column_changed old_value new_value
0         101          price     25.99     23.99
1         105          price      35.0      37.0
   product_id column_changed old_value new_value
0         102          stock       200       180


In [11]:
#displaying the generated list of dataframes
diff_rows

[Empty DataFrame
 Columns: [product_id, column_changed, old_value, new_value]
 Index: [],
 Empty DataFrame
 Columns: [product_id, column_changed, old_value, new_value]
 Index: [],
    product_id column_changed old_value new_value
 0         101          price     25.99     23.99
 1         105          price      35.0      37.0,
    product_id column_changed old_value new_value
 0         102          stock       200       180]

In [12]:
#Appending all the individual dataframes generated from our dynamic sql query ouput
column_changes_df = pd.concat(diff_rows,ignore_index = True).sort_values(by='product_id')

In [13]:
#displaying the results
column_changes_df

,product_id,column_changed,old_value,new_value
0,101,price,25.99,23.99
2,102,stock,200,180
1,105,price,35.0,37.0


In [14]:
#Loading resulted output into csv file
column_changes_df.to_csv('column_level_differences.csv', index=False)